<a href="https://colab.research.google.com/github/MengOonLee/Deep_learning/blob/master/TensorFlow/Customise/SubclassCustom/CustomLayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flexible input shapes for custom layers
In this reading you will learn how to use the build method to allow custom layers to work with flexible sized inputs.

## Fix the input shape in the custom layer

Previously, you have created custom layers by initialising all variables in the `__init__` method. For instance, you defined a dense layer called `MyLayer` as follows:

In [1]:
# Create a custom layer
import tensorflow as tf
tf.random.set_seed(seed=42)

class MyLayer(tf.keras.layers.Layer):
    def __init__(self, units, input_dim, **kwargs):
        super().__init__(**kwargs)
        self.w = self.add_weight(shape=(input_dim, units),
            initializer=tf.keras.initializers.RandomNormal())
        self.b = self.add_weight(shape=(units,),
            initializer=tf.keras.initializers.Zeros())

    def call(self, inputs):
        return tf.linalg.matmul(a=inputs, b=self.w) + self.b

Notice that the required arguments for the `__init__` method are the number of units in the dense layer (`units`) and the input size (`input_dim`). This means that you need to fix these two arguments when you instantiate the layer.

In [2]:
#  Create a custom layer with 3 units and input dimension of 5
my_layer = MyLayer(units=3, input_dim=5)

Since the input size has been fixed to be 5, this custom layer can only take inputs of that size. For example, we can call the layer as follows:

In [3]:
import tensorflow as tf
tf.random.set_seed(seed=42)

# Call the custom layer on a Tensor input of ones
x = tf.random.normal(shape=(1, 5))
print(my_layer(inputs=x))

# Print the layer weights
print(my_layer.weights)

tf.Tensor([[-0.16765237 -0.02140991 -0.09701281]], shape=(1, 3), dtype=float32)
[<KerasVariable shape=(5, 3), dtype=float32, path=my_layer/variable>, <KerasVariable shape=(3,), dtype=float32, path=my_layer/variable_1>]


However, forcing the input shape (and therefore the shape of the weights) to be fixed when the layer is instantiated is unnecessary, and it may be more convenient to only do this later on, after the model has been defined.

For example, in some cases you may not know the input shape at the model building time. We have come across this concept before when building models with the Sequential API. If the `input_shape` argument is omitted, the weights will only be created when an input is passed into the model.

## Allow a flexible input shape in the custom layer

You can delay the weight creation by using the `build` method to define the weights. The `build` method is executed when the `__call__` method is called, meaning the weights are only created only the layer is called with a specific input.

The `build` method has a required argument `input_shape`, which can be used to define the shapes of the layer weights.

In [4]:
import tensorflow as tf
tf.random.set_seed(seed=42)

# Rewrite the custom layer with lazy weight creation
class MyLayer(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super().__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
            initializer=tf.keras.initializers.RandomNormal())
        self.b = self.add_weight(shape=(self.units,),
            initializer=tf.keras.initializers.Zeros())

    def call(self, inputs):
        return tf.linalg.matmul(a=inputs, b=self.w) + self.b

Now, when you instantiate the layer, you only need to specify the number of units in the dense layer (`units`), and not the input size (`input_dim`).

### Create a custom layer with flexible input size

In [5]:
#  Create a custom layer with 3 units
my_layer = MyLayer(units=3)

This layer can now be called on an input of any size, at which point the layer weights will be created and the input size will be fixed.

In [6]:
import tensorflow as tf
tf.random.set_seed(seed=42)

# Call the custom layer on a Tensor input of ones of size 5
x = tf.random.normal(shape=(1, 5))
print(my_layer(inputs=x))

# Print the layer weights
print(my_layer.weights)

tf.Tensor([[-0.16375409 -0.31556225 -0.00555508]], shape=(1, 3), dtype=float32)
[<KerasVariable shape=(5, 3), dtype=float32, path=my_layer_1/variable_2>, <KerasVariable shape=(3,), dtype=float32, path=my_layer_1/variable_3>]


### Create a new custom layer and pass in a different sized input

In [7]:
import tensorflow as tf
tf.random.set_seed(seed=42)

#  Create a new custom layer with 3 units
my_layer = MyLayer(units=3)

# Call the custom layer on a Tensor input of ones of size 4
x = tf.random.normal(shape=(1, 4))
print(my_layer(inputs=x))

# Print the layer weights
print(my_layer.weights)

tf.Tensor([[-0.1050756  -0.07842422 -0.14320967]], shape=(1, 3), dtype=float32)
[<KerasVariable shape=(4, 3), dtype=float32, path=my_layer_2/variable_4>, <KerasVariable shape=(3,), dtype=float32, path=my_layer_2/variable_5>]


Note that the code for creating a custom layer object is identical, but the shape of the weights in the layer depend on the size of the input passed to the layer.

## Flexible input shapes in models

Deferring the weight creation until the layer is called is also useful when using the custom layer as an intermediate layer inside a larger model. In this case you may want to create several custom layer objects in the model, and it is tedious to keep track of the input shape that each of the custom layers needs.

By deferring the weight creation as above, the input shape can be inferred from the output of the previous layer.

In [37]:
import tensorflow as tf
tf.random.set_seed(seed=42)

# Create a model using the custom layer
class MyModel(tf.keras.Model):
    def __init__(self, units_1, units_2, **kwargs):
        super().__init__(**kwargs)
        self.layer_1 = MyLayer(units=units_1)
        self.layer_2 = MyLayer(units=units_2)

    def call(self, inputs):
        x = self.layer_1(inputs=inputs)
        x = tf.nn.relu(features=x)
        x = self.layer_2(inputs=x)
        y = tf.nn.softmax(logits=x)
        return y

In the above model definition, the custom layer `MyLayer` is used twice. Notice that each instance of the custom layer object can have a different input size, depending on the arguments used to create the model and the inputs passed into the model

In [34]:
# Create a custom model object
my_model = MyModel(units_1=32, units_2=10)

We can create and initialise all of the weights of the model by passing in an example Tensor input.

In [38]:
import tensorflow as tf
tf.random.set_seed(seed=42)

inputs =  tf.keras.Input(shape=(100,))
outputs = my_model(inputs=inputs)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Print the model summary
model.summary()

# Create and initialize all of the model weights
model(inputs=tf.random.normal(shape=(1, 100)))

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)          │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ my_model_1 (MyModel)                 │ (None, 10)                  │           3,562 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,562 (13.91 KB)

 Trainable params: 3,562 (13.91 KB)

 Non-trainable params: 0 (0.00 B)

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[0.09334587, 0.10626455, 0.09447167, 0.09360649, 0.10290653,
        0.09062201, 0.09538576, 0.10714776, 0.12737247, 0.08887694]],
      dtype=float32)>

## Further reading and resources
* https://www.tensorflow.org/guide/keras/custom_layers_and_models#best_practice_deferring_weight_creation_until_the_shape_of_the_inputs_is_known